# Notebook 3 · Planner → Executor Agentic RAG

This notebook explores how a lightweight planner can orchestrate multiple retrieval steps. We lean on LangChain's `PlanAndExecute` helper to build a planner that decomposes a question into sub-queries before executing them with our baseline retrieval components.

The pattern is useful for multi-hop or analytical queries where a single search pass is unlikely to surface every fact required for a grounded answer.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI as LangChainChatOpenAI

from pprint import pprint

from shared import (
    DEFAULT_MODEL,
    RetrievalContext,
    build_baseline_chain,
    build_retrieval_context,
    pretty_print_json,
    time_execution,
)


In [ ]:
context = build_retrieval_context(top_k=5)
retriever_tool = create_retriever_tool(
    context.retriever,
    'help_center_search',
    'Use when you need detailed information about NimbusWorkspaces product behaviour, limits, or billing.',
)
planner_llm = LangChainChatOpenAI(model=DEFAULT_MODEL, temperature=0.2)
executor_llm = LangChainChatOpenAI(model=DEFAULT_MODEL, temperature=0.0)

planner = load_chat_planner(planner_llm)
executor = load_agent_executor(executor_llm, [retriever_tool], verbose=True)
plan_and_execute = PlanAndExecute(planner=planner, executor=executor)


In [ ]:
complex_question = (
    'Compare the upgrade paths between the Team and Enterprise plans and list any approval requirements.'
)
response = plan_and_execute({'input': complex_question})
print(response['output'])


## What to look for

* Inspect the console logs above to see the planner's decomposition steps.
* Experiment with higher temperature for the planner to increase exploration.
* Replace the executor with a different tool stack (e.g. web search) to observe how planning adapts.